In [1]:
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
plt.rcParams.update({'font.size': 16})
plt.rcParams['axes.prop_cycle'] = (cycler(color=['tab:blue', 'tab:green', 'tab:orange', 'tab:red']))

sig2fwhm = 2.355

In [26]:
def comp_mos(par):
    foctel = par['ftel'] * par['dtel'] # mm, telescope focal length
    fcoll = par['ftel'] * par['frd'] # collimator F-ratio
    thpix = np.rad2deg(par['frd'] * par['dpix'] / (par['dtel'] * par['fcam'])) * 3600 # arcsec, sky-projected pixel pitch
    dfib = foctel * np.deg2rad(thfib / 3600) # mm, fiber core diameter
    dfib_px = thfib / thpix # px, fiber core diameter
    mag = par['fcam'] / fcoll # spectrograph magnification
    nfib = (par['nspix'] - 2 * par['marg_px']) / (dfib_px + par['dark_px']) # max number of fibers/spectrograph
    pfib = dfib + par['dark_px'] * par['dpix'] / mag # mm, fiber pitch
    lslit = nfib * pfib # mm, max slit length
    fwhm_px = dfib_px * ((3**0.5 / 2)**2 + (par['iq'] * sig2fwhm)**2)**0.5 # px, fiber core FWHM (incl. IQ)
    res = dict(foctel=foctel, fcoll=fcoll, thpix=thpix, dfib=dfib, dfib_px=dfib_px, mag=mag,
               nfib=nfib, pfib=pfib, lslit=lslit, fwhm_px=fwhm_px)
    return res   

In [114]:
def print_res(res, par):
    print('*** spectrograph ***')
    for path in par['paths']:
        lmin, lmax = path
        lcen = (lmin + lmax) / 2
        pix2nm = (lmax - lmin) / (par['ndpix'] - 2 * par['marg_px']) # nm / px
        dl = res['fwhm_px'] * pix2nm
        Rmin = lmin / dl
        Rcen = lcen / dl
        Rmax = lmax / dl
        foct = (lmax - lmin)/(lmin)
        print('Rmin({:.0f} nm) | Rcen({:.0f} nm) | Rmax({:.0f} nm) | Frac Oct | {:.0f} | {:.0f} | {:.0f} | {:.2f}'.format(lmin, lcen, lmax, Rmin, Rcen, Rmax, foct))
    print('spectrograph magnification: {:.2f}'.format(res['mag']))
    print('collimator F-ratio: {:.2f}'.format(res['fcoll']))
    print('camera F-ratio: {:.2f}'.format(par['fcam']))
    print('')


    print('*** telescope parameters ***')
    print('telescope diameter: {:.0f} mm'.format(par['dtel']))
    print('telescope F-ratio: {:.2f}'.format(par['ftel']))
    print('')

    print('*** fiber parameters ***')
    print('fiber core diameter (on-sky | physical | image): {:.2f} arcsec | {:.3f} mm | {:.2f} px'.format(par['thfib'], res['dfib'], res['dfib_px']))
    print('fiber pitch: {:.3f} mm'.format(res['pfib']))
    print('max number of fibers/spectrograph: {:.0f}'.format(res['nfib']))
    print('max slit length: {:.2f} mm'.format(res['lslit']))

    print('number of dark pixels between adjacent fiber cores: {:.1f} px'.format(par['dark_px']))
    print('focal ratio degradation: {:.2f}'.format(par['frd']))
    print('LSF FWHM in pixel: {:.2f}'.format(res['fwhm_px']))
    print('')

    print('*** detector parameters ***')
    print('detector format (spatial x spectral / pitch): {:.0f} x {:.0f} / {:.3f} mm'.format(par['nspix'], par['ndpix'], par['dpix']))
    print('sky-projected pixel pitch: {:.2f} arcsec'.format(res['thpix']))
    print('alignment margin (physical | image): {:.2f} mm | {:.0f} px wide pixel band around detector edges'.format(par['marg_px'] * par['dpix'], par['marg_px']))


In [112]:
def compare_res(reslist, parlist, names):
    print('*** spectrograph ***')
    for res,par,name in zip(reslist,parlist,names):
        for path in par['paths']:
            lmin, lmax = path
            lcen = (lmin + lmax) / 2
            pix2nm = (lmax - lmin) / (par['ndpix'] - 2 * par['marg_px']) # nm / px
            dl = res['fwhm_px'] * pix2nm
            Rmin = lmin / dl
            Rcen = lcen / dl
            Rmax = lmax / dl
            foct = (lmax - lmin)/(lmin)
            print('{} Rmin({:.0f} nm) | Rcen({:.0f} nm) | Rmax({:.0f} nm) | Frac Oct | {:.0f} | {:.0f} | {:.0f} | {:.2f}'.format(name,lmin, lcen, lmax, Rmin, Rcen, Rmax, foct))
        print(' ')   
    txt = [f"{name}: {res['mag']:.2f} " for name,res in zip(names,reslist)]
    print("spectrograph magnification ", *txt)
    txt = [f"{name}: {par['fcoll']:.2f} " for name,par in zip(names,reslist)]
    print("collimator F-ratio ", *txt)
    txt = [f"{name}: {par['fcam']:.2f} " for name,par in zip(names,parlist)]
    print("camera F-ratio ", *txt)
    print('')

    print('*** telescope parameters ***')
    txt = [f"{name}: {par['dtel']/1000:.2f} m " for name,par in zip(names,parlist)]
    print('telescope diameter ', *txt)
    txt = [f"{name}: {par['ftel']:.2f} " for name,par in zip(names,parlist)]
    print('telescope F-ratio ', *txt)
    print('')
    
    print('*** fiber parameters ***')
    txt = [f"{name}: {par['thfib']:.2f} arcsec" for name,par in zip(names,parlist)]
    print('fiber core diameter on-sky ', *txt)
    txt = [f"{name}:  {res['dfib']*1000:.0f} microns " for name,res in zip(names,reslist)]
    print('fiber core diameter physical ', *txt)
    txt = [f"{name}: {res['dfib_px']:.2f} px" for name,res in zip(names,reslist)]
    print('fiber core diameter image ', *txt)
    txt = [f"{name}: {res['pfib']*1000:.0f} microns" for name,res in zip(names,reslist)]
    print('fiber pitch ', *txt)
    txt = [f"{name}: {res['nfib']:.0f} " for name,res in zip(names,reslist)]
    print('max number of fibers/spectrograph ', *txt)
    txt = [f"{name}: {res['lslit']:.1f} mm " for name,res in zip(names,reslist)]
    print('max slit length', *txt)
    txt = [f"{name}: {par['dark_px']:.1f} px " for name,par in zip(names,parlist)]
    print('number of dark pixels between adjacent fiber cores ', *txt)
    txt = [f"{name}: {par['frd']:.2f} " for name,par in zip(names,parlist)]
    print('focal ratio degradation ', *txt)
    txt = [f"{name}: {res['fwhm_px']:.2f} px " for name,res in zip(names,reslist)]
    print('LSF FWHM ', *txt)
    print('')
    
    print('*** detector parameters ***')
    txt = [f"{name}: {par['nspix']}x{par['ndpix']}/{par['dpix']*1000:.0f}mu " for name,par in zip(names,parlist)]
    print('detector format (spatial x spectral / pitch) ', *txt)
    txt = [f"{name}: {res['thpix']:.2f} arcsec " for name,res in zip(names,reslist)]
    print('sky-projected pixel pitch ', *txt)
    txt = [f"{name}: {1000*(par['marg_px'] * par['dpix']):.0f} microns" for name,par in zip(names,parlist)]
    print('alignment margin (physical) ', *txt)
    txt = [f"{name}: {par['marg_px'] * par['marg_px']:.0f} px" for name,par in zip(names,parlist)]
    print('alignment margin (image) ', *txt)
    print('')
    
    print('*** numbers for 20k fibers ***')
    txt = [f"{name}: {20000/res['nfib']:.1f} " for name,res in zip(names,reslist)]
    print('number of spectrograph ', *txt)
    txt = [f"{name}: {len(par['paths'])*20000/res['nfib']:.1f} " for name,res,par in zip(names,reslist,parlist)]
    print('number of detectors ', *txt)



 

# 4MOST-LRS case

Input values from LRS Analysis & Performance Report + MAIT Plan (Issue 4) and 4MOST User Manual (Issue 2)

In [115]:
mospar = dict(
# telescope parameters
dtel = 4100, # mm, telescope diameter
ftel = 3.28, # telescope F-ratio
# fiber parameters
thfib = 1.40, # arcsec, sky-projected fiber core diameter
dark_px = 3.0, # number of dark pixels between adjacent fiber cores -- guessed
marg_px = 20, # px, alignment margin (pixel band around detector edges) -- guessed
frd = 0.92, # focal ratio degradation -- guessed
# spectrograph parameters
paths = [(370, 554),
         (524, 721),
         (691, 950)], # nm, spectral path (lmin, lmax)
fcam = 1.73, # camera F-ratio
iq = 0.10, # spot radius RMS / fiber core image diameter ratio -- guessed
# detector parameters
dpix = 0.015, # mm, pixel pitch
nspix = 6144, # number of pixels along the *spatial* axis
ndpix = 6144, # number of pixels along the *dispersion* axis
)
res = comp_mos(mospar)
print_res(res, mospar)

*** spectrograph ***
Rmin(370 nm) | Rcen(462 nm) | Rmax(554 nm) | Frac Oct | 5488 | 6853 | 8218 | 0.50
Rmin(524 nm) | Rcen(622 nm) | Rmax(721 nm) | Frac Oct | 7260 | 8625 | 9989 | 0.38
Rmin(691 nm) | Rcen(820 nm) | Rmax(950 nm) | Frac Oct | 7282 | 8647 | 10011 | 0.37
spectrograph magnification: 0.57
collimator F-ratio: 3.02
camera F-ratio: 1.73

*** telescope parameters ***
telescope diameter: 4100 mm
telescope F-ratio: 3.28

*** fiber parameters ***
fiber core diameter (on-sky | physical | image): 1.40 arcsec | 0.065 mm | 2.49 px
fiber pitch: 0.144 mm
max number of fibers/spectrograph: 1111
max slit length: 159.71 mm
number of dark pixels between adjacent fiber cores: 3.0 px
focal ratio degradation: 0.92
LSF FWHM in pixel: 2.24

*** detector parameters ***
detector format (spatial x spectral / pitch): 6144 x 6144 / 0.015 mm
sky-projected pixel pitch: 0.40 arcsec
alignment margin (physical | image): 0.30 mm | 20 px wide pixel band around detector edges


# DESI case

Input values from https://arxiv.org/abs/1611.00037

In [116]:
mospar = dict(
# telescope parameters
dtel = 3800, # mm, telescope diameter
ftel = 3.86, # telescope F-ratio
# fiber parameters
thfib = 1.46, # arcsec, sky-projected fiber core diameter
dark_px = 4.0, # number of dark pixels between adjacent fiber cores -- guessed
marg_px = 20, # px, alignment margin (pixel band around detector edges) -- guessed
frd = 0.93, # focal ratio degradation
# spectrograph parameters
paths = [(360, 593),
         (566, 772),
         (747, 980)], # nm, spectral path (lmin, lmax)
fcam = 1.70, # camera F-ratio
iq = 0.20, # spot radius RMS / fiber core image diameter ratio -- guessed
# detector parameters
dpix = 0.015, # mm, pixel pitch
nspix = 4096, # number of pixels along the *spatial* axis
ndpix = 4096, # number of pixels along the *dispersion* axis
)
res = comp_mos(mospar)
print_res(res, mospar)

*** spectrograph ***
Rmin(360 nm) | Rcen(476 nm) | Rmax(593 nm) | Frac Oct | 2831 | 3748 | 4664 | 0.65
Rmin(566 nm) | Rcen(669 nm) | Rmax(772 nm) | Frac Oct | 5035 | 5951 | 6868 | 0.36
Rmin(747 nm) | Rcen(864 nm) | Rmax(980 nm) | Frac Oct | 5875 | 6792 | 7708 | 0.31
spectrograph magnification: 0.47
collimator F-ratio: 3.59
camera F-ratio: 1.70

*** telescope parameters ***
telescope diameter: 3800 mm
telescope F-ratio: 3.86

*** fiber parameters ***
fiber core diameter (on-sky | physical | image): 1.46 arcsec | 0.071 mm | 2.25 px
fiber pitch: 0.198 mm
max number of fibers/spectrograph: 649
max slit length: 128.47 mm
number of dark pixels between adjacent fiber cores: 4.0 px
focal ratio degradation: 0.93
LSF FWHM in pixel: 2.21

*** detector parameters ***
detector format (spatial x spectral / pitch): 4096 x 4096 / 0.015 mm
sky-projected pixel pitch: 0.45 arcsec
alignment margin (physical | image): 0.30 mm | 20 px wide pixel band around detector edges


# WST case (6k x 6k, 15 um detector - 2 paths)

In [129]:
mospar1 = dict(
# telescope parameters
dtel = 11650, # mm, telescope diameter
ftel = 2.8, # telescope F-ratio
# fiber parameters
thfib = 1.0, # arcsec, sky-projected fiber core diameter
dark_px = 4.0, # number of dark pixels between adjacent fiber cores
marg_px = 20, # px, alignment margin (pixel band around detector edges)
frd = 0.95, # focal ratio degradation
# spectrograph parameters
paths = [(350, 620),
         (600, 970)], # nm, spectral path (lmin, lmax)
fcam = 1.0, # camera F-ratio
iq = 0.25, # spot radius RMS / fiber core image diameter ratio
# detector parameters
dpix = 0.015, # mm, pixel pitch
nspix = 6144, # number of pixels along the *spatial* axis
ndpix = 6144 # number of pixels along the *dispersion* axis
)
res1 = comp_mos(mospar1)
print_res(res1, mospar1)

*** spectrograph ***
Rmin(350 nm) | Rcen(485 nm) | Rmax(620 nm) | Frac Oct | 1906 | 2642 | 3377 | 0.77
Rmin(600 nm) | Rcen(785 nm) | Rmax(970 nm) | Frac Oct | 2385 | 3120 | 3855 | 0.62
spectrograph magnification: 0.38
collimator F-ratio: 2.66
camera F-ratio: 1.00

*** telescope parameters ***
telescope diameter: 11650 mm
telescope F-ratio: 2.80

*** fiber parameters ***
fiber core diameter (on-sky | physical | image): 1.00 arcsec | 0.158 mm | 3.96 px
fiber pitch: 0.318 mm
max number of fibers/spectrograph: 766
max slit length: 243.55 mm
number of dark pixels between adjacent fiber cores: 4.0 px
focal ratio degradation: 0.95
LSF FWHM in pixel: 4.15

*** detector parameters ***
detector format (spatial x spectral / pitch): 6144 x 6144 / 0.015 mm
sky-projected pixel pitch: 0.25 arcsec
alignment margin (physical | image): 0.30 mm | 20 px wide pixel band around detector edges


# WST case (4k x 4k, 15 um detector - 4 paths)

In [133]:
mospar2 = dict(
# telescope parameters
dtel = 11650, # mm, telescope diameter
ftel = 2.8, # telescope F-ratio
# fiber parameters
thfib = 1.0, # arcsec, sky-projected fiber core diameter
dark_px = 4.0, # number of dark pixels between adjacent fiber cores
marg_px = 20, # px, alignment margin (pixel band around detector edges)
frd = 0.95, # focal ratio degradation
# spectrograph parameters
paths = [(350, 530),
         (510, 645),
         (625, 790),
         (770, 970)], # nm, spectral path (lmin, lmax)
fcam = 1.0, # camera F-ratio
iq = 0.25, # spot radius RMS / fiber core image diameter ratio
# detector parameters
dpix = 0.015, # mm, pixel pitch
nspix = 4096, # number of pixels along the *spatial* axis
ndpix = 4096, # number of pixels along the *dispersion* axis
)
res2 = comp_mos(mospar2)
print_res(res2, mospar2)

*** spectrograph ***
Rmin(350 nm) | Rcen(440 nm) | Rmax(530 nm) | Frac Oct | 1900 | 2389 | 2877 | 0.51
Rmin(510 nm) | Rcen(578 nm) | Rmax(645 nm) | Frac Oct | 3692 | 4180 | 4669 | 0.26
Rmin(625 nm) | Rcen(708 nm) | Rmax(790 nm) | Frac Oct | 3702 | 4190 | 4679 | 0.26
Rmin(770 nm) | Rcen(870 nm) | Rmax(970 nm) | Frac Oct | 3762 | 4251 | 4739 | 0.26
spectrograph magnification: 0.38
collimator F-ratio: 2.66
camera F-ratio: 1.00

*** telescope parameters ***
telescope diameter: 11650 mm
telescope F-ratio: 2.80

*** fiber parameters ***
fiber core diameter (on-sky | physical | image): 1.00 arcsec | 0.158 mm | 3.96 px
fiber pitch: 0.318 mm
max number of fibers/spectrograph: 509
max slit length: 161.83 mm
number of dark pixels between adjacent fiber cores: 4.0 px
focal ratio degradation: 0.95
LSF FWHM in pixel: 4.15

*** detector parameters ***
detector format (spatial x spectral / pitch): 4096 x 4096 / 0.015 mm
sky-projected pixel pitch: 0.25 arcsec
alignment margin (physical | image): 0.30 m

# WST case (4k x 4k, 15 um detector - 4 paths) with IR

In [135]:
mospar3 = dict(
# telescope parameters
dtel = 11650, # mm, telescope diameter
ftel = 2.8, # telescope F-ratio
# fiber parameters
thfib = 1.0, # arcsec, sky-projected fiber core diameter
dark_px = 4.0, # number of dark pixels between adjacent fiber cores
marg_px = 20, # px, alignment margin (pixel band around detector edges)
frd = 0.95, # focal ratio degradation
# spectrograph parameters
paths = [(370, 530),
         (510, 710),
         (700, 970),
         (960, 1300)], # nm, spectral path (lmin, lmax)
fcam = 1.0, # camera F-ratio
iq = 0.25, # spot radius RMS / fiber core image diameter ratio
# detector parameters
dpix = 0.015, # mm, pixel pitch
nspix = 4096, # number of pixels along the *spatial* axis
ndpix = 4096, # number of pixels along the *dispersion* axis
)
res3 = comp_mos(mospar3)
print_res(res3, mospar3)

*** spectrograph ***
Rmin(370 nm) | Rcen(450 nm) | Rmax(530 nm) | Frac Oct | 2260 | 2748 | 3237 | 0.43
Rmin(510 nm) | Rcen(610 nm) | Rmax(710 nm) | Frac Oct | 2492 | 2980 | 3469 | 0.39
Rmin(700 nm) | Rcen(835 nm) | Rmax(970 nm) | Frac Oct | 2533 | 3022 | 3511 | 0.39
Rmin(960 nm) | Rcen(1130 nm) | Rmax(1300 nm) | Frac Oct | 2759 | 3248 | 3736 | 0.35
spectrograph magnification: 0.38
collimator F-ratio: 2.66
camera F-ratio: 1.00

*** telescope parameters ***
telescope diameter: 11650 mm
telescope F-ratio: 2.80

*** fiber parameters ***
fiber core diameter (on-sky | physical | image): 1.00 arcsec | 0.158 mm | 3.96 px
fiber pitch: 0.318 mm
max number of fibers/spectrograph: 509
max slit length: 161.83 mm
number of dark pixels between adjacent fiber cores: 4.0 px
focal ratio degradation: 0.95
LSF FWHM in pixel: 4.15

*** detector parameters ***
detector format (spatial x spectral / pitch): 4096 x 4096 / 0.015 mm
sky-projected pixel pitch: 0.25 arcsec
alignment margin (physical | image): 0.30

# Compare designs

In [136]:
compare_res([res1,res2,res3],[mospar1,mospar2,mospar3],['mos6k','mos4k','mos4k-IR'])

*** spectrograph ***
mos6k Rmin(350 nm) | Rcen(485 nm) | Rmax(620 nm) | Frac Oct | 1906 | 2642 | 3377 | 0.77
mos6k Rmin(600 nm) | Rcen(785 nm) | Rmax(970 nm) | Frac Oct | 2385 | 3120 | 3855 | 0.62
 
mos4k Rmin(350 nm) | Rcen(440 nm) | Rmax(530 nm) | Frac Oct | 1900 | 2389 | 2877 | 0.51
mos4k Rmin(510 nm) | Rcen(578 nm) | Rmax(645 nm) | Frac Oct | 3692 | 4180 | 4669 | 0.26
mos4k Rmin(625 nm) | Rcen(708 nm) | Rmax(790 nm) | Frac Oct | 3702 | 4190 | 4679 | 0.26
mos4k Rmin(770 nm) | Rcen(870 nm) | Rmax(970 nm) | Frac Oct | 3762 | 4251 | 4739 | 0.26
 
mos4k-IR Rmin(370 nm) | Rcen(450 nm) | Rmax(530 nm) | Frac Oct | 2260 | 2748 | 3237 | 0.43
mos4k-IR Rmin(510 nm) | Rcen(610 nm) | Rmax(710 nm) | Frac Oct | 2492 | 2980 | 3469 | 0.39
mos4k-IR Rmin(700 nm) | Rcen(835 nm) | Rmax(970 nm) | Frac Oct | 2533 | 3022 | 3511 | 0.39
mos4k-IR Rmin(960 nm) | Rcen(1130 nm) | Rmax(1300 nm) | Frac Oct | 2759 | 3248 | 3736 | 0.35
 
spectrograph magnification  mos6k: 0.38  mos4k: 0.38  mos4k-IR: 0.38 
collimato

# Impact of camera f-ratio

In [120]:
mospar3 = mospar1.copy()
mospar3['fcam'] = 1.8
res3 = comp_mos(mospar3)
mospar4 = mospar2.copy()
mospar4['fcam'] = 1.8
res4 = comp_mos(mospar4)
compare_res([res1,res3,res4],[mospar1,mospar3,mospar4],['mos6k-fcam1.0','mos6k-fcam1.8','mos4k-fcam1.8'])

*** spectrograph ***
mos6k-fcam1.0 Rmin(350 nm) | Rcen(485 nm) | Rmax(620 nm) | Frac Oct | 1906 | 2642 | 3377 | 0.77
mos6k-fcam1.0 Rmin(600 nm) | Rcen(785 nm) | Rmax(970 nm) | Frac Oct | 2385 | 3120 | 3855 | 0.62
 
mos6k-fcam1.8 Rmin(350 nm) | Rcen(485 nm) | Rmax(620 nm) | Frac Oct | 1059 | 1468 | 1876 | 0.77
mos6k-fcam1.8 Rmin(600 nm) | Rcen(785 nm) | Rmax(970 nm) | Frac Oct | 1325 | 1733 | 2142 | 0.62
 
mos4k-fcam1.8 Rmin(350 nm) | Rcen(440 nm) | Rmax(530 nm) | Frac Oct | 1056 | 1327 | 1599 | 0.51
mos4k-fcam1.8 Rmin(510 nm) | Rcen(578 nm) | Rmax(645 nm) | Frac Oct | 2051 | 2322 | 2594 | 0.26
mos4k-fcam1.8 Rmin(625 nm) | Rcen(708 nm) | Rmax(790 nm) | Frac Oct | 2056 | 2328 | 2599 | 0.26
mos4k-fcam1.8 Rmin(770 nm) | Rcen(870 nm) | Rmax(970 nm) | Frac Oct | 2090 | 2362 | 2633 | 0.26
 
spectrograph magnification  mos6k-fcam1.0: 0.38  mos6k-fcam1.8: 0.68  mos4k-fcam1.8: 0.68 
collimator F-ratio  mos6k-fcam1.0: 2.66  mos6k-fcam1.8: 2.66  mos4k-fcam1.8: 2.66 
camera F-ratio  mos6k-fcam1.0: 

# Impact of tel f-ratio

In [121]:
mospar3 = mospar1.copy()
mospar3['ftel'] = 1.4
res3 = comp_mos(mospar3)
compare_res([res1,res3],[mospar1,mospar3],['mos6k-ftel2.8','mos6k-ftel1.4'])

*** spectrograph ***
mos6k-ftel2.8 Rmin(350 nm) | Rcen(485 nm) | Rmax(620 nm) | Frac Oct | 1906 | 2642 | 3377 | 0.77
mos6k-ftel2.8 Rmin(600 nm) | Rcen(785 nm) | Rmax(970 nm) | Frac Oct | 2385 | 3120 | 3855 | 0.62
 
mos6k-ftel1.4 Rmin(350 nm) | Rcen(485 nm) | Rmax(620 nm) | Frac Oct | 1906 | 2642 | 3377 | 0.77
mos6k-ftel1.4 Rmin(600 nm) | Rcen(785 nm) | Rmax(970 nm) | Frac Oct | 2385 | 3120 | 3855 | 0.62
 
spectrograph magnification  mos6k-ftel2.8: 0.38  mos6k-ftel1.4: 0.75 
collimator F-ratio  mos6k-ftel2.8: 2.66  mos6k-ftel1.4: 1.33 
camera F-ratio  mos6k-ftel2.8: 1.00  mos6k-ftel1.4: 1.00 

*** telescope parameters ***
telescope diameter  mos6k-ftel2.8: 11.65 m  mos6k-ftel1.4: 11.65 m 
telescope F-ratio  mos6k-ftel2.8: 2.80  mos6k-ftel1.4: 1.40 

*** fiber parameters ***
fiber core diameter on-sky  mos6k-ftel2.8: 1.00 arcsec mos6k-ftel1.4: 1.00 arcsec
fiber core diameter physical  mos6k-ftel2.8:  158 microns  mos6k-ftel1.4:  79 microns 
fiber core diameter image  mos6k-ftel2.8: 3.96 